# Arbitrage Strategy Based on Book-to-Market

This notebook implements an arbitrage model based on the spread between accounting value (book value) and market value of major listed companies.
Enhancements include:
- Sector neutrality
- Transaction cost adjustments
- Combined value signals (Book-to-Market and Free Cash Flow Yield)
- Quality filtering (positive earnings, low accruals)

## Theoretical Framework

Let:
- $P_{i,t}$ = Price of stock *i* at time *t*
- $BVPS_{i,t}$ = Book Value per Share
- $MV_{i,t} = P_{i,t} \times Shares$
- $BV_{i,t} = BVPS_{i,t} \times Shares$

### Value Signal:
We consider book value to market capitalization ratio: how much accounting value is paid for by the market
$$
BTM_{i,t} = \frac{BV_{i,t}}{MV_{i,t}}
$$

### FCF Yield:
$$
FCFY_{i,t} = \frac{FCF_{i,t}}{MV_{i,t}}
$$

### Composite Signal (Z-Score):
$$
Z_{i,t} = \frac{(Signal_{i,t} - \mu_t)}{\sigma_t}
$$
Where $Signal_{i,t}$ combines BTM and FCFY, and $\mu_t$, $\sigma_t$ are cross-sectional mean and std.

We go **long** on stocks with low Z and **short** on high Z, adjusted for sector neutrality.

Should switch to dynamic book value calculation

In [ ]:
# Ideally, only run a few times since key usage is limited

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from utils import get_fundamentals

In [ ]:
# CONFIG FOR AV
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'META', 'AMZN']
TRANSACTION_COST = 0.001  # 0.1% per trade
START_DATE = '2018-01-01'
END_DATE = '2023-12-31'

prices = yf.download(TICKERS, start=START_DATE, end=END_DATE)

fundamentals = {}
for ticker in TICKERS:
    fundamentals[ticker] = get_fundamentals(ticker, function='o') # get current overview

df = pd.DataFrame(fundamentals).T
pd.DataFrame({'Attributes': df.columns.to_list()})

In [ ]:
# Restart
fund_df = pd.DataFrame(fundamentals).T
# Convert to numeric columns that start with a digit
fund_df = fund_df.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.str.contains(r'^\d', na=False).any() else x)

# Filter out companies with negative EPS or profit margin
fund_df = fund_df[(fund_df['DilutedEPSTTM'] > 0) & (fund_df['ProfitMargin'] > 0)]

fund_df = fund_df[['BookValue', 'SharesOutstanding', 'EPS', 'DilutedEPSTTM', 'MarketCapitalization',
             'Sector', 'ProfitMargin','PriceToBookRatio']]

fund_df['Book to Market1'] = 1 / fund_df['PriceToBookRatio']  # Book to Market ratio 1
fund_df['Book to Market2'] = (fund_df['BookValue'] * fund_df['SharesOutstanding']) / fund_df['MarketCapitalization']
#fund_df.insert(0, 'Book to Market', fund_df.pop('Book to Market'))

# Oddly enough, the 'Book to Market1' and 'Book to Market2' columns are NOT identical.
# What differences in source data are driving this? To be investigated.
fund_df.iloc[0]

BookValue                       4.471
SharesOutstanding         14935800000
EPS                              6.41
DilutedEPSTTM                    6.41
MarketCapitalization    3194468958000
Sector                     TECHNOLOGY
ProfitMargin                    0.243
PriceToBookRatio                47.82
Book to Market1              0.020912
Book to Market2              0.020904
Name: AAPL, dtype: object

In [ ]:
close_prices = prices['Close']

book_values, fcf_yields, accruals, market_values, sectors, earnings = {}, {}, {}, {}, {}, {}

TICKERS=['AAPL']

for ticker in TICKERS:
    if ticker not in fund_df.index:
        continue  # Skip tickers not in the fund_df
    _ticker = yf.Ticker(ticker)
    info = _ticker.info
    cf = _ticker.cashflow
    bs = _ticker.balance_sheet

    
    shares_outstanding = info.get("sharesOutstanding")
    book_value = info.get("bookValue") * shares_outstanding
    pe_ratio = info.get("trailingPE")
    sector = info.get("sector")
    
    fcf = cf.loc['Free Cash Flow']
    earnings[ticker] = cf.loc['Net Income From Continuing Operations']
    op_cash = cf.loc['Operating Cash Flow']

    assets = bs.loc['Total Assets']
    
    market_cap = close_prices[ticker].values * shares_outstanding
    market_values[ticker] = market_cap
    book_values[ticker] = book_value
    fcf_yields[ticker] = fcf / market_cap.mean() if market_cap.mean() > 0 else np.nan
    accruals[ticker] = (earnings[ticker] - op_cash) / assets
    sectors[ticker] = sector

market_values = pd.DataFrame(market_values)
book_values = pd.DataFrame(pd.Series(book_values))
btm = book_values.T.iloc[0] / market_values # Because book_values is currently STATIC
btm = btm.replace([np.inf, -np.inf], np.nan).dropna()
z_btm = (btm - btm.mean(axis=1).values[:, None]) / btm.std(axis=1).values[:, None]
z_btm = z_btm.fillna(0) # No std-dev, so fill with 0

sector_map = pd.Series(sectors)
z_sector_adj = z_btm.copy()
for date in z_btm.index:
    for sector in sector_map.unique():
        tickers_in_sector = sector_map[sector_map == sector].index
        if len(tickers_in_sector) >= 2:
            sector_mean = z_btm.loc[date, tickers_in_sector].mean()
            z_sector_adj.loc[date, tickers_in_sector] -= sector_mean

# Apply quality filters
quality = pd.Series(accruals).apply(lambda x: abs(x) < 0.25) & (earnings[ticker] > 0)
filtered_tickers = quality[quality].index
signal = z_sector_adj[filtered_tickers].shift(1)
signal.index = close_prices[filtered_tickers].index
returns = close_prices[filtered_tickers].pct_change().loc[signal.index]

daily_return = (signal * returns).mean(axis=1)
gross_return = (1 + daily_return).cumprod()
net_return = daily_return - TRANSACTION_COST * 0.10
net_cum_return = (1 + net_return).cumprod()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(gross_return, label='Gross Return')
plt.plot(net_cum_return, label='Net Return (with TC)', linestyle='--')
plt.title("Arbitrage Strategy Cumulative Returns")
plt.xlabel("Date")
plt.ylabel("Portfolio Value")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()